In [25]:
# Import dependencies
from sleeperpy import User, Leagues
import pandas as pd

In [72]:
# User inputs username and selects a league from landing page
username = "dirtywizard"
# Get User's Sleeper ID and Sleeper leagues
user_id = User.get_user(username)['user_id']
leagues = Leagues.get_all_leagues(user_id, 'nfl', 2024)
league_name_list = [x['name'] for x in leagues]
# Show the user the list of names and have them select one
print(league_name_list)
league_name = "No Coffee Sharps D.F.F.L."
# Find the league id for the selected league
league_id = [x['league_id'] for x in leagues if x['name'] == league_name][0]

['No Coffee Sharps D.F.F.L.']


In [73]:
# Receive user ID & league ID in GET request, then fetch league data from Sleeper API
selected_league = Leagues.get_league(league_id)
roster_positions = selected_league['roster_positions']
scoring_settings = selected_league['scoring_settings']
# Get the rosters for the league
rosters = Leagues.get_rosters(league_id)
users = Leagues.get_users(league_id)
# Get a list of all players currently on rosters
rostered_players = [x['players'] for x in rosters]
rostered_players = [item for sublist in rostered_players for item in sublist]

In [74]:
# Read RDS database 
# Select id_sleeper, full_name, age, team, number, position from players where id_sleeper in rostered_players
allplayers = pd.read_csv('test_data/rds_players.csv') 
# Read RDS database
# Select * from predictions where id_sleeper in rostered_players
predictions = pd.read_csv('test_data/rds_scoring.csv')
predictions = predictions.loc[predictions['id_sleeper'].isin(rostered_players)]
predictions = predictions.merge(allplayers[['id_sleeper', 'full_name', 'age', 'team', 'number', 'position']], on='id_sleeper', how='left')

/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_80323/647561162.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  predictions = pd.read_csv('test_data/rds_scoring.csv')


In [75]:
# Multiply each prediction column by the corresponding scoring setting
for column in predictions.columns:
    if column in scoring_settings:
        predictions[column] = predictions[column] * scoring_settings[column]
# Sum up the total points for each column in the scoring settings
predictions['predicted_fantasy_points'] = predictions[list(scoring_settings.keys())].sum(axis=1)
predictions = predictions.drop(columns=list(scoring_settings.keys()))
predictions.sort_values(by='predicted_fantasy_points', ascending=False, inplace=True)
predictions

,index_scoring,id_sleeper,week_of_season,full_name,age,team,number,position,predicted_fantasy_points
340,7523_0,7523,0,Trevor Lawrence,24.0,JAX,16.0,QB,19.48
350,7523_10,7523,10,Trevor Lawrence,24.0,JAX,16.0,QB,18.39
465,6768_7,6768,7,Tua Tagovailoa,26.0,MIA,1.0,QB,18.12
349,7523_9,7523,9,Trevor Lawrence,24.0,JAX,16.0,QB,18.04
348,7523_8,7523,8,Trevor Lawrence,24.0,JAX,16.0,QB,17.87
...,...,...,...,...,...,...,...,...,...
1170,DET_13,DET,13,Detroit Lions,NaN,DET,NaN,DEF,-2.20
1082,CHI_10,CHI,10,Chicago Bears,NaN,CHI,NaN,DEF,-2.58
1089,CHI_17,CHI,17,Chicago Bears,NaN,CHI,NaN,DEF,-2.58
1397,IND_1,IND,1,Indianapolis Colts,NaN,IND,NaN,DEF,-3.86
